
http://proproprogs.ru/neural_network/kak-rekurrentnaya-neyronnaya-set-prognoziruet-simvoly


У нас задача предсказать четвёртый символ.

In [1]:
import re


In [3]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [6]:
len(text)

205

In [7]:
num_characters = 34 #33 буквы + пробел

In [8]:
import keras
from keras.preprocessing.text import Tokenizer #для токенизации обуч.выборки

In [9]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)#cоздаём обуч.выборку,которую
#надо токенизировать
#cоздаём экз класса tokenizer
#максимальное количество слов (символов), которое вернет Tokenizer (если
#элементов будет больше, то останутся наиболее повторяющиеся в тексте):

In [10]:
#Tokenizer мы создали,теперь надо его обучить
#И пропустим через него загруженный текст
tokenizer.fit_on_texts(text)

In [12]:
print(tokenizer.word_index) # И формируется словарь,где каждому символу поставлен в соответствие
#свой уникальный индекс.В дальнейшем нам это пригодится, когда мы прогнозируемое значение будем
#переводить в символ.

#если на вход подаём 0,то и на выходе будет 0


{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [13]:
#Далее, преобразуем текст в набор ОНЕ-векторов:
inp_chars = 3 # мы хотим предсказывать 4-е слово
data = tokenizer.texts_to_matrix(text)
#здесь символы упорядочены не по алфавиту, а в соответствии со словарем tokenizer.word_index.
#Затем, из этой матрицы мы сформируем тензор обучающей выборки и соответствующий набор выходных значений. Для начала вычислим размер обучающего
#множества:

In [14]:
data[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [26]:
n = data.shape[0]-inp_chars # -3
n  #размер обучающего множества

202

In [17]:
import numpy as np
#И, далее, сформируем входной тензор и прогнозные значения.


In [18]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа


In [19]:
len(X) # в Х мы сгенерировали 202 обучающих примера

202

In [24]:
X[0] #вход

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [25]:
Y[0] #выход

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
from keras.layers import *
from keras.models import Sequential

In [28]:
#Данные для обучения готовы. Теперь создадим рекуррентную НС с помощью Keras в
#соответствии с архитектурой рисунка выше:
model = Sequential()
model.add(Input((inp_chars, num_characters))) #размер входной инф - 3, 34
model.add(SimpleRNN(500, activation='tanh')) #реккурентный слой лучше всего работает с тангенсом
model.add(Dense(num_characters, activation='softmax')) # только одна 1 на каком-то нейроне на выходе из 34
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 500)               267500    
                                                                 
 dense_2 (Dense)             (None, 34)                17034     
                                                                 
Total params: 284534 (1.09 MB)
Trainable params: 284534 (1.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
7/7 [==============================] - 1s 14ms/step - loss: 3.4296 - accuracy: 0.0941
Epoch 2/100
7/7 [==============================] - 0s 11ms/step - loss: 2.7781 - accuracy: 0.2921
Epoch 3/100
7/7 [==============================] - 0s 11ms/step - loss: 2.3835 - accuracy: 0.2921
Epoch 4/100
7/7 [==============================] - 0s 12ms/step - loss: 2.1431 - accuracy: 0.3465
Epoch 5/100
7/7 [==============================] - 0s 11ms/step - loss: 1.9197 - accuracy: 0.3911
Epoch 6/100
7/7 [==============================] - 0s 12ms/step - loss: 1.7531 - accuracy: 0.4505
Epoch 7/100
7/7 [==============================] - 0s 12ms/step - loss: 1.6211 - accuracy: 0.5099
Epoch 8/100
7/7 [==============================] - 0s 12ms/step - loss: 1.5024 - accuracy: 0.5050
Epoch 9/100
7/7 [==============================] - 0s 15ms/step - loss: 1.3806 - accuracy: 0.5495
Epoch 10/100
7/7 [==============================] - 0s 12ms/step - loss: 1.2928 - accuracy: 0.5792
Epoch 11/100
7/7 [=

In [31]:
#Далее, объявим вспомогательную функцию, в которой будет выполняться прогноз очередного
#символа и добавления его в конец начальной строки:

def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)#с помощью reshape делаем табличку 3 на 34

    pred = model.predict( inp ) # предсказываем 4 символ
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str

In [35]:
res = buildPhrase("утренн")
print(res)

1/1 [==============================] - 0s 20ms/step
утреннрте  тлвоувчзлерьйчытеот лвнрачю тетхпввнре с туоп


Почему такие непонятные слова - потому что 3 символа всего было задействовано.
Строка - 50 символов.

Этот текст сгенерирован посимвольно. Сеть даже пробелы расставила, а потом вышла в установившийся режим с повторяемыми фразами.
Этот пример больше для ознакомления, чтобы показать, как кодируется текстовая информация, как формируется обучающая выборка и как строится рекуррентная сеть в Keras.

# Слова


In [37]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [39]:
maxWordsCount = 1000  #размер словаря, вектор из 1000 элементов(слов)
#Каждое слово, затем, будет кодироваться one-hot вектором в соответствии с его номером в словаре:
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
#filters - исключаемые из текста символы (по умолчанию, следующие: !-»—#$%&()*+,-./:<→>?
#@N^-El3~\t\n\r), lower = True - автоматический перевод в нижний регистр для единообразия больших и
#малых символов,char_level=False - если False, то текст делится на слова, иначе - на символы.

In [42]:
#Tokenizer обучили и на выходе-словарь
dist = list(tokenizer.word_counts.items())
print(dist[:10])
#Здесь отображаются кортежи со словом и его частотой встречаемости в тексте.

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [43]:
data = tokenizer.texts_to_sequences([texts])
#Далее, мы преобразуем текст в последовательность чисел в соответствии с полученным словарем.
#Для этого используется специальный метод класса Tokenizer.

In [49]:
data #На выходе получим массив чисел объекта numpy:


[[1,
  4,
  5,
  6,
  7,
  8,
  9,
  2,
  3,
  10,
  11,
  12,
  13,
  2,
  14,
  15,
  16,
  17,
  18,
  19,
  1,
  20,
  2,
  3,
  21,
  22,
  23,
  24,
  1,
  25,
  26]]

In [50]:
len(data[0])

31

In [53]:
#Осталось закодировать числа массива data в one-hot векторы. Для этого мы воспользуемся методом
# to_categorical пакета Keras:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )
#каждое слово заменили вектором на 1000 чисел, но единица там будет одна

(31, 1000)


In [54]:
inp_words = 3 # ПРИ ОБУЧЕНИИ БУДЕМ УЧИТЫВАТЬ 3 ПРЕДЫДУЩИХ СЛОВА
n = res.shape[0]-inp_words

In [55]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [58]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax')) #ТОЛЬКО СЕЙЧАС НА ВЫХОДЕ 1000 нейронов(maxWordsCount)
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_5 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_5 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
history = model.fit(X, Y, batch_size=32, epochs=19)

Epoch 1/19
1/1 [==============================] - 1s 1s/step - loss: 6.9082 - accuracy: 0.0000e+00
Epoch 2/19
1/1 [==============================] - 0s 23ms/step - loss: 6.8824 - accuracy: 0.0000e+00
Epoch 3/19
1/1 [==============================] - 0s 19ms/step - loss: 6.8564 - accuracy: 0.0000e+00
Epoch 4/19
1/1 [==============================] - 0s 25ms/step - loss: 6.8300 - accuracy: 0.1786
Epoch 5/19
1/1 [==============================] - 0s 18ms/step - loss: 6.8028 - accuracy: 0.4286
Epoch 6/19
1/1 [==============================] - 0s 18ms/step - loss: 6.7747 - accuracy: 0.6429
Epoch 7/19
1/1 [==============================] - 0s 18ms/step - loss: 6.7453 - accuracy: 0.8214
Epoch 8/19
1/1 [==============================] - 0s 19ms/step - loss: 6.7144 - accuracy: 0.8929
Epoch 9/19
1/1 [==============================] - 0s 22ms/step - loss: 6.6815 - accuracy: 0.8929
Epoch 10/19
1/1 [==============================] - 0s 20ms/step - loss: 6.6462 - accuracy: 0.8929
Epoch 11/19
1/1 [==

In [103]:
#Запишем функцию для формирования текста из спрогнозированных слов:

def buildPhrase(texts, str_len = 20):
  res = texts
  data = tokenizer.texts_to_sequences([texts])[0]
  for i in range(str_len):
    x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)

    res += " " + tokenizer.index_word[indx] # дописываем строку

  return res


In [126]:
data = tokenizer. texts_to_sequences ([texts])[0]

In [105]:
data

[1,
 4,
 5,
 6,
 7,
 8,
 9,
 2,
 3,
 10,
 11,
 12,
 13,
 2,
 14,
 15,
 16,
 17,
 18,
 19,
 1,
 20,
 2,
 3,
 21,
 22,
 23,
 24,
 1,
 25,
 26]

In [127]:
for i in range(1):
    x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount)

In [128]:
inp = x.reshape(1, inp_words, maxWordsCount)

In [129]:
inp

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [130]:
len(x) #x-это категоризация первых 3-х слов

3

In [131]:
#И вызовем ее с тремя первыми словами:
res = buildPhrase("позитив добавляет годы")
print(res)

ValueError: cannot reshape array of size 0 into shape (1,3,1000)